In [2]:
import os
import numpy as np
import xlrd

%run helperfx.py

In [3]:
def ConvertMedToModelData(filename, CasVar, MaltoVar):
    
    # Extracts lick data form medfiles
    data = medfilereader(filename, varsToExtract=[CasVar, MaltoVar], remove_var_header=True)
    LicksCas = data[0]
    LicksMalt = data[1]

    # Combines left and right licks
    LicksAll = np.asarray(LicksCas+LicksMalt)
    LicksAll_code = np.asarray([1]*len(LicksCas) + [0]*len(LicksMalt))

    # orders licks in time and sorts coded lick train
    inds = np.argsort(LicksAll)

    LicksAll_sorted = LicksAll[inds]
    LicksAll_code_sorted = LicksAll_code[inds]

    # uses ILIs to find gaps between clusters
    LicksAll_diff = np.diff(LicksAll_sorted)
    gaps = [i for i, x in enumerate(LicksAll_diff) if x > 0.5]

    # Adds zeros to produce gaps between clusters
    gaps.reverse()
    for idx in gaps:
        LicksAll_code_sorted = np.insert(LicksAll_code_sorted, idx+1, 5)

    # Inserts a zero at the start of lick train to match model data
    LicksAll_code_sorted = np.insert(LicksAll_code_sorted, 0, 5)
    
    # changes 5s to 0.5 as arrays are integer only
    output = [0.5 if x==5 else x for x in LicksAll_code_sorted]

    return output

In [4]:
xlfile = '..\\data\\IPP3_expdetails.xlsx'
metafilename = 'IPP3_metafile'

metafilemaker(xlfile, metafilename, fileformat='txt')

metadata, header = metafilereader(metafilename+'.txt')

In [6]:
medfolder = '..\\data\\IPP3_medfiles\\'

test1_data_NR = {}
test1_data_PR = {}

for row in metadata:
    if row[2] == 's4':
        rat = row[1]
        filename = medfolder + row[0]
        
        if 'cas' in row[6]:
            casvar = 'b'
            maltvar = 'e'
        if 'cas' in row[7]:
            casvar = 'e'
            maltvar = 'b'        
        
        output = ConvertMedToModelData(filename, casvar, maltvar)
        
        if row[4] == 'NR': 
            test1_data_NR[rat] = output
        if row[4] == 'PR': 
            test1_data_PR[rat] = output

In [7]:
# for i in test1_data_PR:
#     print(test1_data_PR[i])
#     print(len([x for x in test1_data_PR[i] if x == 1]))

In [8]:
print('Extracted licks from Med data')

Extracted licks from Med data
